In [1]:
import win32com.client as win32
import os

# WIZZAIR


In [31]:
temp_dir = "C:\\Temp"
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
    
save_dir = "C:\\Users\\Kamil Tomaszek\\Desktop\\OREX\\repo\\wizzair\\mail"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
outlook = win32.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")
inbox = namespace.GetDefaultFolder(6)
messages = inbox.Items

for message in messages:
    if message.Subject.startswith("FW: FAKTURY WIZ"):
        attachments = message.Attachments
        for attachment in attachments:
            if attachment.FileName.endswith('.msg'):
                # Use a unique filename to avoid conflicts
                temp_path = os.path.join(temp_dir, f"{attachment.FileName}")
                
                try:
                    # Save the .msg attachment to a temporary location
                    attachment.SaveAsFile(temp_path)
                    
                    # Open the .msg file as a new mail item
                    new_mail = outlook.CreateItemFromTemplate(temp_path)
                    
                    # Iterate through the attachments of the new mail item
                    for sub_attachment in new_mail.Attachments:
                        if sub_attachment.FileName.endswith('.pdf'):
                            pdf_path = os.path.join(save_dir, sub_attachment.FileName)
                            sub_attachment.SaveAsFile(pdf_path)
                            print(f"Saved PDF: {pdf_path}")
                except Exception as e:
                    print(f"Failed to process attachment {attachment.FileName}: {e}")

# wyciągi

In [39]:
import zipfile as zip
from pdfminer.high_level import extract_text
from re import search

In [23]:
dirr = "C:\\Users\\Kamil Tomaszek\\Desktop\\OREX\\repo\\wyciagi"
if not os.path.exists(dirr):
    os.makedirs(dirr)

In [ ]:
import win32com.client as win32

outlook = win32.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")
inbox = namespace.GetDefaultFolder(6)
messages = inbox.Items

for message in messages:
    if message.Subject.find("WB") != -1:
        attachments = message.Attachments
        for attachment in attachments:
            print(f"Attachment: {attachment.FileName}")

In [60]:
for folder in namespace.Folders:
    for subfolder in folder.Folders:
        for subsubfolder in subfolder.Folders:
            for subsubsubfolder in subsubfolder.Folders:
                if subsubsubfolder.Name == 'wyciągi bankowe':
                    wszystkie = subsubsubfolder.Items

In [ ]:
# for message in messages:
# for message in wszystkie:
    print(f"        {message.Subject}")
    for attachment in message.Attachments:
        if attachment.FileName.endswith('.pdf'):
            attachment.SaveAsFile(os.path.join(dirr, attachment.FileName))
            print(f"            {attachment.FileName}")
        if attachment.FileName.endswith('.zip'):
            # Save the .zip attachment to a temporary location
            attachment.SaveAsFile(os.path.join(temp_dir, attachment.FileName))
            with zip.ZipFile(os.path.join(temp_dir, attachment.FileName), 'r') as z:
                z.extractall(dirr)
                print('extracted ', attachment.FileName)

In [62]:
DATE_PATTERN = r'\b\d{2}\.\d{2}\.\d{4}\b'
def extract_date_from_pdf(file_path: str) -> str:
    text = extract_text(file_path)
    match = search(DATE_PATTERN, text)
    if match:
        return match.group(0)
    else:
        return None

In [63]:
def move_file(file: str, how: int, folder):
    if how == 2:
        # date is the last 8 characters of the filename before .pdf in the format YYYYMMDD
        date = file[-12:-4]
        year = date[:4]
        month = date[4:6]
        day = date[6:]
        accnum = r'(\d{4})(PLN|EUR|USD)'
        account = search(accnum, file).group(1)
        month_folder = year + '-' + month
        day_folder = year + '-' + month + '-' + day
        if not os.path.exists(os.path.join(folder, month_folder)):
            os.makedirs(os.path.join(folder, month_folder))
        if not os.path.exists(os.path.join(folder, month_folder, day_folder)):
            os.makedirs(os.path.join(folder, month_folder, day_folder))
        new_file = account + '.pdf'
        os.rename(os.path.join(folder, file), os.path.join(folder, month_folder, day_folder, 'S' + new_file))
    if how == 1:
        date = extract_date_from_pdf(os.path.join(folder, file))
        year = date[-4:]
        month = date[3:5]
        day = date[:2]
        month_folder = year + '-' + month
        day_folder = year + '-' + month + '-' + day
        if not os.path.exists(os.path.join(folder, month_folder)):
            os.makedirs(os.path.join(folder, month_folder))
        if not os.path.exists(os.path.join(folder, month_folder, day_folder)):
            os.makedirs(os.path.join(folder, month_folder, day_folder))
        accnum = file[18:22]
        new_file = accnum + '.pdf'
        try:
            os.rename(os.path.join(folder, file), os.path.join(folder, month_folder, day_folder, 'B' + new_file))
        except:
            print('Error: ', file)


In [64]:
for file in os.listdir(dirr):
    if file.endswith('.pdf'):
        if file.startswith('PL'):
            move_file(file, 2, dirr)
        else:
            move_file(file, 1, dirr)